In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

In [ ]:
device = torch.device('cpu')

In [ ]:
class FaceMaskDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224,224))

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
class FaceMaskDetector:
    def __init__(self, model_path):
        self.device = torch.device('cpu')
        self.model = self.load_model(model_path).to(self.device)
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def load_model(self, model_path):
        model = models.resnet50(pretrained=False)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 2)
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        model.eval()
        return model
    def detect(self, frame):
        frame= self.transform(frame).unsqueeze(0).to(self.device)
        with torch.no_grad():
            output = self.model(frame)
        probabilities = torch.softmax(output, dim=1)[0]
        mask_probability = probabilities[1].item()
        no_mask_probability = probabilities[0].item()
        label = 'Mask' if mask_probability > no_mask_probability else "No Mask"
        return label

In [ ]:
train = "./Face Mask Dataset/Train/"
validation = "./Face Mask Dataset/Validation/"
test = "./Face Mask Dataset/Test/"
model_save_path = "./"
batch_size = 16
num_epochs = 1

In [ ]:
train_images_paths = []
train_labels = []

for folder_name in os.listdir(train):
    folder_path = os.path.join(train, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".png"):
                file_path = os.path.join(folder_path, file_name)
                train_images_paths.append(file_path)
                train_labels.append(0 if folder_name =='WithMask' else 1)

In [ ]:
validation_images_paths = []
validation_labels = []

for folder_name in os.listdir(validation):
    folder_path = os.path.join(validation, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".png"):
                file_path = os.path.join(folder_path, file_name)
                train_images_paths.append(file_path)
                train_labels.append(0 if folder_name =='WithMask' else 1)

In [ ]:
test_images_paths = []
test_labels = []

for folder_name in os.listdir(test):
    folder_path = os.path.join(test, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".png"):
                file_path = os.path.join(folder_path, file_name)
                train_images_paths.append(file_path)
                train_labels.append(0 if folder_name =='WithMask' else 1)

In [ ]:
train_dataset = FaceMaskDataset(train_images_paths, train_labels, transform=transforms.ToTensor())
val_dataset = FaceMaskDataset(validation_images_paths, validation_labels, transform=transforms.ToTensor())
test_dataset =FaceMaskDataset(test_images_paths, test_labels, transform=transforms.ToTensor())

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class FaceMaskModel(nn.Module):
    def __init__(self):
        super(FaceMaskModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.model(x)

In [ ]:
model = FaceMaskModel()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += (predicted == labels).sum().item()
        running_loss += loss.item()
    
    train_accuracy = correct/total
    train_loss = running_loss/len(train_loader)


    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += loss.item()
    val_accuracy = val_correct/val_total
    val_loss = val_loss/len(validation_loader)

    print(f"Epoch {epoch +1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train accuracy: {train_accuracy*100:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | val accuracy: {val_accuracy*100:.2f}%")

    

In [ ]:
torch.save(model.state_dict(), model_save_path)

model.eval()
test_loss = 0.0
test_correct =0
test_total=0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        _,predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        test_loss += loss.item()
test_accuracy = test_correct/test_total
test_loss = test_loss/len(test_loader)

print(f"Test loss:{test_loss:.4f} | Test accuracy: {test_accuracy:.2f}%")

torch.save(model.state_dict(), model_save_path)
torch.save(model, 'faceMaskDetection.h5')



In [ ]:
model = torch.load('faceMaskDetection.h5')
model.eval() 


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_face_mask(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = face_roi / 255.0
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_tensor = torch.tensor(face_roi, dtype=torch.float32)
        with torch.no_grad():
            output = model(face_tensor)
            _, predicted = torch.max(output, 1)
        label = "Mask" if predicted.item() == 1 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
    return frame

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = detect_face_mask(frame)
    cv2.imshow('Face Mask Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

